In [ ]:
# import required packages
from alpha_vantage.timeseries import TimeSeries
import json
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
from datetime import datetime
import logging
from kafka.structs import TopicPartition
import os.path

In [ ]:
# messages to Kafka Server
topic = 'Enter Topic Name'
bootstrap_servers = 'localhost:9092'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda x:json.dumps(x).encode('utf-8'))

In [ ]:
# initialize last timestamp with a required starting value
last_timestamp=datetime.strptime('2021-01-01 00:00:00', "%Y-%m-%d %H:%M:%S")

"""
Recursively goes through the Alpha Vantage API stock data after waiting for 5 minutes
Upload new data arrived using AV API & updating recording TimeStamp
"""

import time
# Define key & TimeSeries object to pull stock data using Alpha Vantage API
key = 'Enter API Key'
ts = TimeSeries(key, output_format='json')
while True:
    try:
        # get data using get_intraday function of AV API
        data, meta = ts.get_intraday('Enter stock symbol', interval='1min', outputsize='full')

        # set stock data in chronological order
        lst=sorted(data)
        data2={}
        for l in lst:
            data2[l]=data[l]
        # send data to Kafka server for which data point data is not sent
        for key in data2:
            rec_date=datetime.strptime(key, "%Y-%m-%d %H:%M:%S")
            print(rec_date)
            if rec_date > last_timestamp:
                dic={}
                dic[key]=data[key]
                producer.send(topic=topic, value=dic)
                last_timestamp=rec_date
                print(rec_date)
        # updating last time stamp in related topic time stamp file
        file=open(topic+'_timestamp.txt','w')
        file.write(str(last_timestamp))
        file.close()
        time.sleep(300)
    except Exception as e:
        logging.warning('exception while pulling datapoints from API / sending data to Kafka')

In [ ]:
# close Kafka producer & consumer objects used
producer.close()
consumer.close()